# (14.0) Описание решаемой задачи

На основании заявок, данные по которым были отражены в ордерлоге к моменту времени T необходимо восстановить очередь активных заявок ( = список лимитных заявок, которые ожидают исполнения), сформировать стакан и рассчитать его метрики:
1. лучшие цены
2. объем на уровне лучших цен
3. спред

# (14.1) Установка необходимых пакетов

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# (14.2) Определение функций¶

In [3]:
def ReverseToDateTime(t):
    format = '%Y%m%d%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

# (14.3) Список всех использующихся датафреймов¶

In [4]:
# orders - ордерлог на заданную дату
# instr_orders - ордерлог на заданную дату по выбранному инструменту на момет времени t
# plc_orders - выставленные заявки
# wdr_orders - отмененные заявки
# trd_orders - исполненные заявки
# lob_almost - очередь заявок

# (14.4) Чтение данных¶

In [5]:
# создаем список из всех названий файлов с данными
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [6]:
# создаем список из инструментов, относящихся к числу голубых фишек
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

In [7]:
# определение всех необходимых данных для расчета
orderlog_name = orderlogs[0]
d = orderlog_name[8:16]
ticker = tickers[0]
time = '110000000'

In [9]:
%%time
# чтение ордерлога
orders = pd.read_csv(orderlogs[0], header = 0)

orders

CPU times: user 5.21 s, sys: 1.15 s, total: 6.36 s
Wall time: 7.6 s


,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,AFKS,S,100000000000,1,1,10.057,100,NaN,NaN
1,2,ABRD,S,100000000000,2,1,150.000,500,NaN,NaN
2,3,ABRD,S,100000000000,3,1,139.500,30,NaN,NaN
3,4,AFLT,S,100000000000,4,1,102.740,10,NaN,NaN
4,5,AGRO,B,100000000000,5,1,786.000,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5424366,5424367,CBOM,S,184459255003,40385,0,5.558,41800,NaN,NaN
5424367,5424368,CBOM,S,184459255367,40399,0,5.562,55300,NaN,NaN
5424368,5424369,CBOM,S,184459255755,40407,0,5.566,104600,NaN,NaN
5424369,5424370,PIKK,B,184459256100,1007823,0,353.000,1250,NaN,NaN


In [56]:
%%time
# фильтруем данные по инструменту
instr_orders = orders[(orders.SECCODE == ticker)]
instr_orders = instr_orders.reset_index()  

instr_orders

CPU times: user 490 ms, sys: 25.9 ms, total: 516 ms
Wall time: 523 ms


,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,53,54,GAZP,B,100000000000,54,1,150.00,10,NaN,NaN
1,58,59,GAZP,B,100000000000,59,1,151.59,200,NaN,NaN
2,65,66,GAZP,B,100000000000,66,1,152.00,10,NaN,NaN
3,70,71,GAZP,B,100000000000,71,1,154.99,10,NaN,NaN
4,431,432,GAZP,S,100000000000,432,1,170.11,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
346742,5424218,5424219,GAZP,S,184429740974,2198823,0,156.00,40,NaN,NaN
346743,5424219,5424220,GAZP,S,184432098651,2326756,0,155.37,1000,NaN,NaN
346744,5424268,5424269,GAZP,S,184451327893,2331971,0,155.50,10,NaN,NaN
346745,5424281,5424282,GAZP,S,184457439758,2073027,0,157.45,1000,NaN,NaN


In [57]:
%%time
# добавляем столбец с датой и преобразуем время в формат datetime
instr_orders ['DATE'] = d

instr_orders['together_time'] = instr_orders['DATE'].apply(str)+instr_orders['TIME'].apply(str)
instr_orders['TIME'] = [ReverseToDateTime(instr_orders.together_time[i]) for i in range (len(instr_orders.together_time))]
del instr_orders['together_time']

instr_orders

CPU times: user 17 s, sys: 270 ms, total: 17.3 s
Wall time: 18 s


,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE
0,53,54,GAZP,B,2019-03-04 10:00:00.000000,54,1,150.00,10,NaN,NaN,20190304
1,58,59,GAZP,B,2019-03-04 10:00:00.000000,59,1,151.59,200,NaN,NaN,20190304
2,65,66,GAZP,B,2019-03-04 10:00:00.000000,66,1,152.00,10,NaN,NaN,20190304
3,70,71,GAZP,B,2019-03-04 10:00:00.000000,71,1,154.99,10,NaN,NaN,20190304
4,431,432,GAZP,S,2019-03-04 10:00:00.000000,432,1,170.11,10,NaN,NaN,20190304
...,...,...,...,...,...,...,...,...,...,...,...,...
346742,5424218,5424219,GAZP,S,2019-03-04 18:44:29.740974,2198823,0,156.00,40,NaN,NaN,20190304
346743,5424219,5424220,GAZP,S,2019-03-04 18:44:32.098651,2326756,0,155.37,1000,NaN,NaN,20190304
346744,5424268,5424269,GAZP,S,2019-03-04 18:44:51.327893,2331971,0,155.50,10,NaN,NaN,20190304
346745,5424281,5424282,GAZP,S,2019-03-04 18:44:57.439758,2073027,0,157.45,1000,NaN,NaN,20190304


In [54]:
%%time
# фильтруем данные по времени
together_time = ReverseToDateTime(d + time)
instrtime_orders = instr_orders[(instr_orders.TIME <= together_time)]

instrtime_orders

CPU times: user 9.52 ms, sys: 16.1 ms, total: 25.7 ms
Wall time: 25.3 ms


,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE
0,53,54,GAZP,B,2019-03-04 10:00:00.000000,54,1,150.00,10,NaN,NaN,20190304
1,58,59,GAZP,B,2019-03-04 10:00:00.000000,59,1,151.59,200,NaN,NaN,20190304
2,65,66,GAZP,B,2019-03-04 10:00:00.000000,66,1,152.00,10,NaN,NaN,20190304
3,70,71,GAZP,B,2019-03-04 10:00:00.000000,71,1,154.99,10,NaN,NaN,20190304
4,431,432,GAZP,S,2019-03-04 10:00:00.000000,432,1,170.11,10,NaN,NaN,20190304
...,...,...,...,...,...,...,...,...,...,...,...,...
43210,985134,985135,GAZP,S,2019-03-04 10:59:59.303931,515421,1,157.64,20,NaN,NaN,20190304
43211,985240,985241,GAZP,B,2019-03-04 10:59:59.449403,513069,0,155.94,10000,NaN,NaN,20190304
43212,985261,985262,GAZP,S,2019-03-04 10:59:59.451286,513076,0,156.33,10000,NaN,NaN,20190304
43213,985314,985315,GAZP,B,2019-03-04 10:59:59.455894,515488,1,155.87,10000,NaN,NaN,20190304


# (14.5) Формирование очереди заявок

In [51]:
%%time 
# устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
no_mkt_orders = instrtime_orders['PRICE']!=0
# таблица выставленных заявок
plc_orders = instrtime_orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(instrtime_orders['ACTION']==1) & no_mkt_orders]
# таблица отмененных заявок
wdr_orders = instrtime_orders [['ORDERNO','VOLUME']][(instrtime_orders['ACTION'] == 0) & no_mkt_orders]
# таблица исполненных заявок
trd_orders = instrtime_orders[['ORDERNO','VOLUME']][(instrtime_orders['ACTION']==2) & no_mkt_orders]
trd_orders = trd_orders[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
trd_orders['ORDERNO'] = trd_orders.index
trd_orders.index=range(trd_orders.shape[0])
# собираем все типы заявок в один датафрейм - очередь заявок
lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
lob_almost = lob_almost.fillna(0)
lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
# удаляем из очереди заявок заявки с нулевым или отрицательным объемом
lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]

lob_almost

CPU times: user 60.2 ms, sys: 16.6 ms, total: 76.8 ms
Wall time: 85.5 ms


,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME_trd,TOTAL_VOL
0,54,B,150.00,10,0.0,0.0,10.0
2,66,B,152.00,10,0.0,0.0,10.0
3,71,B,154.99,10,0.0,0.0,10.0
4,432,S,170.11,10,0.0,0.0,10.0
5,440,B,150.10,200,0.0,0.0,200.0
...,...,...,...,...,...,...,...
21989,515288,B,155.98,100,0.0,0.0,100.0
21990,515314,B,155.89,2600,0.0,0.0,2600.0
21992,515421,S,157.64,20,0.0,0.0,20.0
21993,515488,B,155.87,10000,0.0,0.0,10000.0


# (14.6) Восстановление стакана

In [53]:
%%time
lob = lob_almost[lob_almost['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')

lob

CPU times: user 21.2 ms, sys: 2.82 ms, total: 24 ms
Wall time: 26.3 ms


BUYSELL,B,S
PRICE,,
143.10,1410.0,NaN
143.11,210.0,NaN
143.12,150.0,NaN
143.19,150.0,NaN
143.22,270.0,NaN
...,...,...
170.00,NaN,28740.0
170.07,NaN,11800.0
170.09,NaN,60.0


# (14.7) Расчет метрик ликвидности

In [16]:
lob.sort_index(axis = 0, inplace = True, ascending = False)
lob = lob.fillna(0)

lob

BUYSELL,B,S
PRICE,,
170.11,0.0,8540.0
170.10,0.0,30.0
170.09,0.0,60.0
170.07,0.0,11800.0
170.00,0.0,28740.0
...,...,...
143.22,270.0,0.0
143.19,150.0,0.0
143.12,150.0,0.0


In [45]:
# расчет лучшей цены на покупку
bid_price = lob[lob['B'] > 0].index[0]
# расчет объема на уровне лучшей цены на покупку
bid_volume = lob['B'][bid_price]
# расчет лучшей цены на продажу
ask_price = lob[lob['S'] > 0].index[-1]
# расчет объема на уровне лучшей цены на продажу
ask_volume = lob['S'][ask_price]
# расчет спреда
spread = ask_price - bid_price

print(bid_price, bid_volume, ask_price, ask_volume, spread)

156.07 6170.0 156.11 3900.0 0.040000000000020464


In [36]:
bid_price = lob[lob['B'] > 0].index[0]
bid_price

156.07

In [49]:
liquidity = []

In [50]:
liquidity.append([bid_price, bid_volume, ask_price, ask_volume, spread])

liquidity

[[156.07, 6170.0, 156.11, 3900.0, 0.040000000000020464]]

# (14.8) Создаем цикл для расчета метрик ликвидности на каждые 5 минут

In [61]:
%%time
# определяем дату, инструмент
date = '20190304'
ticker = 'ALRS'
# отбираем данные для расчета
orders = pd.read_csv(orderlogs[0], header = 0)
instr_orders = orders[(orders.SECCODE == ticker)].reset_index() 
instr_orders ['DATE'] = d
instr_orders['together_time'] = instr_orders['DATE'].apply(str)+instr_orders['TIME'].apply(str)
instr_orders['TIME'] = [ReverseToDateTime(instr_orders.together_time[i]) for i in range (len(instr_orders.together_time))]
del instr_orders['together_time']

instr_orders

CPU times: user 15.8 s, sys: 1.32 s, total: 17.1 s
Wall time: 17.4 s


,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE
0,240,241,ALRS,S,2019-03-04 10:00:00.000000,241,1,99.10,550,NaN,NaN,20190304
1,254,255,ALRS,S,2019-03-04 10:00:00.000000,255,1,98.85,33870,NaN,NaN,20190304
2,383,384,ALRS,S,2019-03-04 10:00:00.000000,384,1,99.76,50,NaN,NaN,20190304
3,388,389,ALRS,S,2019-03-04 10:00:00.000000,389,1,98.24,50,NaN,NaN,20190304
4,391,392,ALRS,B,2019-03-04 10:00:00.000000,392,1,92.61,1000,NaN,NaN,20190304
...,...,...,...,...,...,...,...,...,...,...,...,...
213755,5424123,5424124,ALRS,S,2019-03-04 18:44:02.288707,2546224,0,93.73,10,NaN,NaN,20190304
213756,5424144,5424145,ALRS,B,2019-03-04 18:44:09.387865,2479540,0,93.04,500,NaN,NaN,20190304
213757,5424145,5424146,ALRS,B,2019-03-04 18:44:09.469972,2479570,0,93.01,500,NaN,NaN,20190304
213758,5424253,5424254,ALRS,S,2019-03-04 18:44:45.150707,1591060,0,94.17,370,NaN,NaN,20190304


In [80]:
%%time 
# определяем начальный момент времени
time = ReverseToDateTime('20190304'+'100500000000')
liquidity = []

while time <= ReverseToDateTime('20190304'+'184500000000'): # определяем конечный момент времени
    
    # восстанавливаем книгу лимитированных заявок
    # отбираем данные в соответствии с правой границей времени
    instrtime_orders = instr_orders[(instr_orders.TIME <= time)]
    # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mkt_orders = instrtime_orders['PRICE']!=0
    # таблица выставленных заявок
    plc_orders = instrtime_orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(instrtime_orders['ACTION']==1) & no_mkt_orders]
    # таблица отмененных заявок
    wdr_orders = instrtime_orders [['ORDERNO','VOLUME']][(instrtime_orders['ACTION'] == 0) & no_mkt_orders]
    # таблица исполненных заявок
    trd_orders = instrtime_orders[['ORDERNO','VOLUME']][(instrtime_orders['ACTION']==2) & no_mkt_orders]
    trd_orders = trd_orders[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
    trd_orders['ORDERNO'] = trd_orders.index
    trd_orders.index=range(trd_orders.shape[0])
    # собираем все типы заявок в один датафрейм - очередь заявок
    lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
    lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
    lob_almost = lob_almost.fillna(0)
    lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
    # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]
    
    # строим стакан
    lob = lob_almost[lob_almost['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    # расчет лучшей цены на покупку
    bid_price = lob[lob['B'] > 0].index[0]
    # расчет объема на уровне лучшей цены на покупку
    bid_volume = lob['B'][bid_price]
    # расчет лучшей цены на продажу
    ask_price = lob[lob['S'] > 0].index[-1]
    # расчет объема на уровне лучшей цены на продажу
    ask_volume = lob['S'][ask_price]
    # расчет спреда
    spread = ask_price - bid_price
    
    # записываем полученные результаты в список
    liquidity.append([time, bid_price, bid_volume, ask_price, ask_volume, spread])
    print(time, bid_price, bid_volume, ask_price, ask_volume, spread)
    
    time = time + dt.timedelta(minutes=5)

liquidity

2019-03-04 10:05:00 94.98 430.0 95.08 410.0 0.09999999999999432
2019-03-04 10:10:00 95.0 1200.0 95.05 550.0 0.04999999999999716
2019-03-04 10:15:00 95.12 400.0 95.15 60.0 0.030000000000001137
2019-03-04 10:20:00 95.15 3260.0 95.21 580.0 0.05999999999998806
2019-03-04 10:25:00 95.29 750.0 95.3 3000.0 0.009999999999990905
2019-03-04 10:30:00 95.4 2360.0 95.44 310.0 0.03999999999999204
2019-03-04 10:35:00 95.65 480.0 95.73 400.0 0.0799999999999983
2019-03-04 10:40:00 95.74 750.0 95.77 2430.0 0.030000000000001137
2019-03-04 10:45:00 95.66 1010.0 95.69 10.0 0.030000000000001137
2019-03-04 10:50:00 95.64 480.0 95.65 940.0 0.010000000000005116
2019-03-04 10:55:00 95.54 50.0 95.56 1900.0 0.01999999999999602
2019-03-04 11:00:00 95.46 170.0 95.48 1030.0 0.020000000000010232
2019-03-04 11:05:00 95.42 510.0 95.43 1520.0 0.010000000000005116
2019-03-04 11:10:00 95.34 80.0 95.41 1760.0 0.06999999999999318
2019-03-04 11:15:00 95.16 340.0 95.19 520.0 0.030000000000001137
2019-03-04 11:20:00 95.12 570.

KeyboardInterrupt: 

# (14.9) Создаем цикл для расчета метрик ликвидности по всем инструментам на каждые 5 минут внутри одного дня

In [84]:
%%time
# определяем дату
date = '20190304'
# отбираем данные для расчета
orders = pd.read_csv(orderlogs[0], header = 0)
print(1)
orders ['DATE'] = d
orders['together_time'] = orders['DATE'].apply(str) + orders['TIME'].apply(str)
orders['TIME'] = [ReverseToDateTime(orders.together_time[i]) for i in range (len(orders.together_time))]
del orders['together_time']
print(2)

liquidity = []

for ticker in tickers:
    
    ticker = ticker # определяем инструмент, для которого осуществляется расчет
    instr_orders = orders[(orders.SECCODE == ticker)].reset_index() # отбираем данные по инструменту
    time = ReverseToDateTime('20190304'+'100500000000') # определяем начальный момент времени
    
    while time <= ReverseToDateTime('20190304'+'184500000000'): # определяем конечный момент времени
    
        # восстанавливаем книгу лимитированных заявок
        # отбираем данные в соответствии с правой границей времени
        instrtime_orders = instr_orders[(instr_orders.TIME <= time)]
        # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
        no_mkt_orders = instrtime_orders['PRICE']!=0
        # таблица выставленных заявок
        plc_orders = instrtime_orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(instrtime_orders['ACTION']==1) & no_mkt_orders]
        # таблица отмененных заявок
        wdr_orders = instrtime_orders [['ORDERNO','VOLUME']][(instrtime_orders['ACTION'] == 0) & no_mkt_orders]
        # таблица исполненных заявок
        trd_orders = instrtime_orders[['ORDERNO','VOLUME']][(instrtime_orders['ACTION']==2) & no_mkt_orders]
        trd_orders = trd_orders[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
        trd_orders['ORDERNO'] = trd_orders.index
        trd_orders.index=range(trd_orders.shape[0])
        # собираем все типы заявок в один датафрейм - очередь заявок
        lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
        lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
        lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
        lob_almost = lob_almost.fillna(0)
        lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
        # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
        lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]

        # строим стакан
        lob = lob_almost[lob_almost['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
        lob.sort_index(axis = 0, inplace = True, ascending = False)
        lob = lob.fillna(0)
        # расчет лучшей цены на покупку
        bid_price = lob[lob['B'] > 0].index[0]
        # расчет объема на уровне лучшей цены на покупку
        bid_volume = lob['B'][bid_price]
        # расчет лучшей цены на продажу
        ask_price = lob[lob['S'] > 0].index[-1]
        # расчет объема на уровне лучшей цены на продажу
        ask_volume = lob['S'][ask_price]
        # расчет спреда
        spread = ask_price - bid_price

        # записываем полученные результаты в список
        liquidity.append([time, ticker, bid_price, bid_volume, ask_price, ask_volume, spread])

        time = time + dt.timedelta(minutes=5)
        
    print(ticker, bid_price, bid_volume, ask_price, ask_volume, spread)


1
2
GAZP 154.66 250.0 154.69 4680.0 0.030000000000001137
SBER 205.63 730.0 205.7 11000.0 0.06999999999999318
GMKN 13834.0 2.0 13842.0 11.0 8.0
LKOH 5543.5 37.0 5547.5 1.0 4.0
MTSS 255.65 100.0 255.95 20.0 0.29999999999998295
MGNT 3861.0 144.0 3862.5 36.0 1.5
TATN 771.4 50.0 772.0 40.0 0.6000000000000227
NVTK 1091.0 10.0 1093.4 10.0 2.400000000000091
YNDX 2303.6 42.0 2304.0 1.0 0.40000000000009095
ROSN 399.05 30.0 399.3 1260.0 0.25
FIVE 1664.5 10.0 1669.0 75.0 4.5
VTBR 0.03596 80000.0 0.03599 1100000.0 3.0000000000002247e-05
SNGS 25.43 7300.0 25.45 1200.0 0.019999999999999574
CHMF 1042.6 30.0 1043.8 10.0 1.2000000000000455
ALRS 93.61 20.0 93.7 430.0 0.09000000000000341
CPU times: user 8min 48s, sys: 28.7 s, total: 9min 17s
Wall time: 8min 41s


In [1]:
liquidity1 = pd.DataFrame(liquidity)
liquidity1.rename(columns={0: 'TIME', 1: 'SECCODE', 2: 'BID_PRICE', 3: 'BID_VOLUME', 4: 'ASK_PRICE', 5: 'ASK_VOLUME', 6: 'SPREAD'}, inplace=True)
liquidity1

NameError: name 'pd' is not defined

# (14.10) Создаем цикл для расчета метрик ликвидности по всем инструментам на каждые 5 минут для всех дней

In [14]:
%%time
liquidity = []

for l in orderlogs:
    
    print(0)
    date = l[8:16]
    orders = pd.read_csv(l, header = 0)
    print(1)
    orders ['DATE'] = date
    orders['together_time'] = orders['DATE'].apply(str) + orders['TIME'].apply(str)
    orders['TIME'] = [ReverseToDateTime(orders.together_time[i]) for i in range (len(orders.together_time))]
    del orders['together_time']
    print(2)
    
    for ticker in tickers:
    
        ticker = ticker # определяем инструмент, для которого осуществляется расчет
        instr_orders = orders[(orders.SECCODE == ticker)].reset_index() # отбираем данные по инструменту
        time = ReverseToDateTime(date + '100500000000') # определяем начальный момент времени
    
        while time <= ReverseToDateTime(date + '184500000000'): # определяем конечный момент времени
    
            # восстанавливаем книгу лимитированных заявок
            # отбираем данные в соответствии с правой границей времени
            instrtime_orders = instr_orders[(instr_orders.TIME <= time)]
            # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
            no_mkt_orders = instrtime_orders['PRICE']!=0
            # таблица выставленных заявок
            plc_orders = instrtime_orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(instrtime_orders['ACTION']==1) & no_mkt_orders]
            # таблица отмененных заявок
            wdr_orders = instrtime_orders [['ORDERNO','VOLUME']][(instrtime_orders['ACTION'] == 0) & no_mkt_orders]
            # таблица исполненных заявок
            trd_orders = instrtime_orders[['ORDERNO','VOLUME']][(instrtime_orders['ACTION']==2) & no_mkt_orders]
            trd_orders = trd_orders[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
            trd_orders['ORDERNO'] = trd_orders.index
            trd_orders.index=range(trd_orders.shape[0])
            # собираем все типы заявок в один датафрейм - очередь заявок
            lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
            lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
            lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
            lob_almost = lob_almost.fillna(0)
            lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
            # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
            lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]

            # строим стакан
            lob = lob_almost[lob_almost['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
            lob.sort_index(axis = 0, inplace = True, ascending = False)
            lob = lob.fillna(0)
            # расчет лучшей цены на покупку
            bid_price = lob[lob['B'] > 0].index[0]
            # расчет объема на уровне лучшей цены на покупку
            bid_volume = lob['B'][bid_price]
            # расчет лучшей цены на продажу
            ask_price = lob[lob['S'] > 0].index[-1]
            # расчет объема на уровне лучшей цены на продажу
            ask_volume = lob['S'][ask_price]
            # расчет спреда
            spread = ask_price - bid_price

            # записываем полученные результаты в список
            liquidity.append([time, date, ticker, bid_price, bid_volume, ask_price, ask_volume, spread])

            time = time + dt.timedelta(minutes = 5)
        
        print(time, date, ticker, bid_price, bid_volume, ask_price, ask_volume, spread)

0
1
2
2019-03-04 18:50:00 20190304 GAZP 154.66 250.0 154.69 4680.0 0.030000000000001137
2019-03-04 18:50:00 20190304 SBER 205.63 730.0 205.7 11000.0 0.06999999999999318
2019-03-04 18:50:00 20190304 GMKN 13834.0 2.0 13842.0 11.0 8.0
2019-03-04 18:50:00 20190304 LKOH 5543.5 37.0 5547.5 1.0 4.0
2019-03-04 18:50:00 20190304 MTSS 255.65 100.0 255.95 20.0 0.29999999999998295
2019-03-04 18:50:00 20190304 MGNT 3861.0 144.0 3862.5 36.0 1.5
2019-03-04 18:50:00 20190304 TATN 771.4 50.0 772.0 40.0 0.6000000000000227
2019-03-04 18:50:00 20190304 NVTK 1091.0 10.0 1093.4 10.0 2.400000000000091
2019-03-04 18:50:00 20190304 YNDX 2303.6 42.0 2304.0 1.0 0.40000000000009095
2019-03-04 18:50:00 20190304 ROSN 399.05 30.0 399.3 1260.0 0.25
2019-03-04 18:50:00 20190304 FIVE 1664.5 10.0 1669.0 75.0 4.5
2019-03-04 18:50:00 20190304 VTBR 0.03596 80000.0 0.03599 1100000.0 3.0000000000002247e-05
2019-03-04 18:50:00 20190304 SNGS 25.43 7300.0 25.45 1200.0 0.019999999999999574
2019-03-04 18:50:00 20190304 CHMF 1042.

2019-03-14 18:50:00 20190314 TATN 729.3 1600.0 730.8 140.0 1.5
2019-03-14 18:50:00 20190314 NVTK 1059.4 60.0 1060.6 110.0 1.199999999999818
2019-03-14 18:50:00 20190314 YNDX 2303.0 12.0 2305.4 30.0 2.400000000000091
2019-03-14 18:50:00 20190314 ROSN 398.8 14970.0 399.0 40.0 0.19999999999998863
2019-03-14 18:50:00 20190314 FIVE 1613.5 2.0 1617.0 5.0 3.5
2019-03-14 18:50:00 20190314 VTBR 0.03611 11710000.0 0.03612 12140000.0 9.999999999996123e-06
2019-03-14 18:50:00 20190314 SNGS 24.21 600.0 24.25 100.0 0.03999999999999915
2019-03-14 18:50:00 20190314 CHMF 1005.2 20.0 1005.6 10.0 0.39999999999997726
2019-03-14 18:50:00 20190314 ALRS 95.51 1260.0 95.61 880.0 0.09999999999999432
0
1
2
2019-03-15 18:50:00 20190315 GAZP 153.3 310.0 153.39 3000.0 0.08999999999997499
2019-03-15 18:50:00 20190315 SBER 203.9 60280.0 203.99 1300.0 0.09000000000000341
2019-03-15 18:50:00 20190315 GMKN 14188.0 8.0 14198.0 5.0 10.0
2019-03-15 18:50:00 20190315 LKOH 5626.5 200.0 5633.5 100.0 7.0
2019-03-15 18:50:00 2

2019-03-25 18:50:00 20190325 FIVE 1618.0 572.0 1621.5 17.0 3.5
2019-03-25 18:50:00 20190325 VTBR 0.036375 10040000.0 0.036405 21390000.0 3.0000000000002247e-05
2019-03-25 18:50:00 20190325 SNGS 24.515 28400.0 24.54 10000.0 0.02499999999999858
2019-03-25 18:50:00 20190325 CHMF 1009.0 700.0 1009.8 130.0 0.7999999999999545
2019-03-25 18:50:00 20190325 ALRS 95.61 4650.0 95.77 20.0 0.1599999999999966
0
1
2
2019-03-26 18:50:00 20190326 GAZP 151.37 250.0 151.46 1000.0 0.09000000000000341
2019-03-26 18:50:00 20190326 SBER 217.83 190.0 217.9 1770.0 0.06999999999999318
2019-03-26 18:50:00 20190326 GMKN 14288.0 213.0 14300.0 18.0 12.0
2019-03-26 18:50:00 20190326 LKOH 5802.5 1.0 5805.0 1869.0 2.5
2019-03-26 18:50:00 20190326 MTSS 262.5 2830.0 263.45 4200.0 0.9499999999999886
2019-03-26 18:50:00 20190326 MGNT 3759.0 102.0 3765.0 1.0 6.0
2019-03-26 18:50:00 20190326 TATN 756.9 10.0 758.3 50.0 1.3999999999999773
2019-03-26 18:50:00 20190326 NVTK 1079.2 30.0 1081.4 20.0 2.2000000000000455
2019-03-26 

In [15]:
liquidity1 = pd.DataFrame(liquidity)
liquidity1.rename(columns={'0': 'TIME', '1': 'DATE', '2': 'SECCODE', '3': 'BID_PRICE', '4': 'BID_VOLUME', '5': 'ASK_PRICE', '6': 'ASK_VOLUME', '7': 'SPREAD'}, inplace=True)

liquidity1

,0,1,2,3,4,5,6,7
0,2019-03-04 10:05:00,20190304,GAZP,156.38,610.0,156.40,2880.0,0.02
1,2019-03-04 10:10:00,20190304,GAZP,156.26,22160.0,156.30,800.0,0.04
2,2019-03-04 10:15:00,20190304,GAZP,156.36,10.0,156.40,2290.0,0.04
3,2019-03-04 10:20:00,20190304,GAZP,156.10,2210.0,156.16,2430.0,0.06
4,2019-03-04 10:25:00,20190304,GAZP,156.30,410.0,156.36,460.0,0.06
...,...,...,...,...,...,...,...,...
31495,2019-03-29 18:25:00,20190329,ALRS,92.27,10.0,92.28,19150.0,0.01
31496,2019-03-29 18:30:00,20190329,ALRS,92.49,3020.0,92.51,1320.0,0.02
31497,2019-03-29 18:35:00,20190329,ALRS,92.58,15410.0,92.59,34590.0,0.01
31498,2019-03-29 18:40:00,20190329,ALRS,92.56,1240.0,92.59,4500.0,0.03


In [29]:
liquidity1

,TIME,DATE,SECCODE,BID_PRICE,BID_VOLUME,ASK_PRICE,ASK_VOLUME,SPREAD
0,04.03.2019 10:05,20190304,GAZP,156.38,610,156.40,2880,0.02
1,04.03.2019 10:10,20190304,GAZP,156.26,22160,156.30,800,0.04
2,04.03.2019 10:15,20190304,GAZP,156.36,10,156.40,2290,0.04
3,04.03.2019 10:20,20190304,GAZP,156.10,2210,156.16,2430,0.06
4,04.03.2019 10:25,20190304,GAZP,156.30,410,156.36,460,0.06
...,...,...,...,...,...,...,...,...
29920,29.03.2019 18:25,20190329,ALRS,92.27,10,92.28,19150,0.01
29921,29.03.2019 18:30,20190329,ALRS,92.49,3020,92.51,1320,0.02
29922,29.03.2019 18:35,20190329,ALRS,92.58,15410,92.59,34590,0.01
29923,29.03.2019 18:40,20190329,ALRS,92.56,1240,92.59,4500,0.03


# (14.11) Группируем айсберг-заявки по пятиминутным интервалам

In [31]:
icebergs = pd.read_csv('iceberg_new.csv', header = 0)

icebergs

,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,GROUP_ID,TOTAL_VOLUME
0,0,B,20190304,1,199077,156.28,GAZP,100643595004,100936467866,-2830,2000,NaN,NaN
1,1,B,20190304,1,359638,156.18,GAZP,103027072289,105331759856,-210,2000,NaN,NaN
2,2,S,20190304,1,397852,156.58,GAZP,103657193855,103805393784,-3000,2000,NaN,NaN
3,3,B,20190304,1,486370,156.18,GAZP,105331809856,105342904115,-1000,1790,NaN,NaN
4,4,B,20190304,1,516105,156.03,GAZP,110003906212,110005256841,-1000,2000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54501,436,B,20190329,1,3821455,92.47,ALRS,183219963550,183219963586,-360,930,NaN,NaN
54502,437,B,20190329,1,3845898,92.58,ALRS,183514255473,183514256529,-1000,1000,NaN,NaN
54503,438,B,20190329,1,3845899,92.58,ALRS,183514255473,183514256529,-490,1000,NaN,NaN
54504,439,B,20190329,1,3845904,92.58,ALRS,183514257134,183514257134,-770,510,NaN,NaN


In [107]:
icebergs.dtypes

index             int64
BUYSELL          object
DATE              int64
ICEBERG           int64
ORDERNO           int64
PRICE           float64
SECCODE          object
TIME              int64
TIME_IDENT        int64
VOLUME            int64
VOLUME_INI        int64
GROUP_ID        float64
TOTAL_VOLUME    float64
dtype: object

In [32]:
icebergs['together_time'] = icebergs['DATE'].apply(str) + icebergs['TIME'].apply(str)
icebergs['together_time_ident'] = icebergs['DATE'].apply(str) + icebergs['TIME_IDENT'].apply(str)
icebergs['TIME'] = [ReverseToDateTime(icebergs.together_time[i]) for i in range (len(icebergs.together_time))]
icebergs['TIME_IDENT'] = [ReverseToDateTime(icebergs.together_time_ident[i]) for i in range (len(icebergs.together_time_ident))]
del icebergs['together_time']
del icebergs['together_time_ident']

In [114]:
icebergs['DATE'] = icebergs['DATE'].apply(str)
icebergs.dtypes

index                    int64
BUYSELL                 object
DATE                    object
ICEBERG                  int64
ORDERNO                  int64
PRICE                  float64
SECCODE                 object
TIME            datetime64[ns]
TIME_IDENT      datetime64[ns]
VOLUME                   int64
VOLUME_INI               int64
GROUP_ID               float64
TOTAL_VOLUME           float64
dtype: object

In [33]:
%%time
ice_per_5min = []
icebergs = icebergs

for l in orderlogs:
    
    d = l[8:16]
    
    for m in tickers:
        
        ticker = m
        iceberg_instr = icebergs[(icebergs.SECCODE == ticker) & (icebergs.DATE == d)]
        time_end = ReverseToDateTime(d + '100500000000') # определяем начальный момент времени
        
        while time_end <= ReverseToDateTime(d + '184500000000'): # определяем конечный момент времени
            
            time_begin = time_end - dt.timedelta(minutes=5)
            ice_count = iceberg_instr[(iceberg_instr.TIME >= time_begin) & (iceberg_instr.TIME <= time_end)]['SECCODE'].count()
            ice_vol = iceberg_instr[(iceberg_instr.TIME >= time_begin) & (iceberg_instr.TIME <= time_end)]['VOLUME_INI'].sum() - iceberg_instr[(iceberg_instr.TIME >= time_begin) & (iceberg_instr.TIME <= time_end)]['VOLUME'].sum()
            ice_flag = 0
            
            if ice_count != 0:
                ice_flag = 1
            
            ice_per_5min.append ([time_end, d, ticker, ice_flag, ice_count])
            print (time_end, d, ticker, ice_flag, ice_count, ice_vol)
            
            time_end = time_end + dt.timedelta(minutes=5)

ice_per_5min1 = pd.DataFrame(ice_per_5min)
ice_per_5min1.rename(columns={0: 'TIME', 1: 'DATE', 2: 'SECCODE', 3: 'FLAG', 4: 'ICE_COUNT', 5: 'ICE_VOL'}, inplace=True)
ice_per_5min1

/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


2019-03-04 10:05:00 20190304 GAZP 0 0 0
2019-03-04 10:10:00 20190304 GAZP 0 0 0
2019-03-04 10:15:00 20190304 GAZP 0 0 0
2019-03-04 10:20:00 20190304 GAZP 0 0 0
2019-03-04 10:25:00 20190304 GAZP 0 0 0
2019-03-04 10:30:00 20190304 GAZP 0 0 0
2019-03-04 10:35:00 20190304 GAZP 0 0 0
2019-03-04 10:40:00 20190304 GAZP 0 0 0
2019-03-04 10:45:00 20190304 GAZP 0 0 0
2019-03-04 10:50:00 20190304 GAZP 0 0 0
2019-03-04 10:55:00 20190304 GAZP 0 0 0
2019-03-04 11:00:00 20190304 GAZP 0 0 0
2019-03-04 11:05:00 20190304 GAZP 0 0 0
2019-03-04 11:10:00 20190304 GAZP 0 0 0
2019-03-04 11:15:00 20190304 GAZP 0 0 0
2019-03-04 11:20:00 20190304 GAZP 0 0 0
2019-03-04 11:25:00 20190304 GAZP 0 0 0
2019-03-04 11:30:00 20190304 GAZP 0 0 0
2019-03-04 11:35:00 20190304 GAZP 0 0 0
2019-03-04 11:40:00 20190304 GAZP 0 0 0
2019-03-04 11:45:00 20190304 GAZP 0 0 0
2019-03-04 11:50:00 20190304 GAZP 0 0 0
2019-03-04 11:55:00 20190304 GAZP 0 0 0
2019-03-04 12:00:00 20190304 GAZP 0 0 0
2019-03-04 12:05:00 20190304 GAZP 0 0 0


2019-03-04 18:25:00 20190304 SBER 0 0 0
2019-03-04 18:30:00 20190304 SBER 0 0 0
2019-03-04 18:35:00 20190304 SBER 0 0 0
2019-03-04 18:40:00 20190304 SBER 0 0 0
2019-03-04 18:45:00 20190304 SBER 0 0 0
2019-03-04 10:05:00 20190304 GMKN 0 0 0
2019-03-04 10:10:00 20190304 GMKN 0 0 0
2019-03-04 10:15:00 20190304 GMKN 0 0 0
2019-03-04 10:20:00 20190304 GMKN 0 0 0
2019-03-04 10:25:00 20190304 GMKN 0 0 0
2019-03-04 10:30:00 20190304 GMKN 0 0 0
2019-03-04 10:35:00 20190304 GMKN 0 0 0
2019-03-04 10:40:00 20190304 GMKN 0 0 0
2019-03-04 10:45:00 20190304 GMKN 0 0 0
2019-03-04 10:50:00 20190304 GMKN 0 0 0
2019-03-04 10:55:00 20190304 GMKN 0 0 0
2019-03-04 11:00:00 20190304 GMKN 0 0 0
2019-03-04 11:05:00 20190304 GMKN 0 0 0
2019-03-04 11:10:00 20190304 GMKN 0 0 0
2019-03-04 11:15:00 20190304 GMKN 0 0 0
2019-03-04 11:20:00 20190304 GMKN 0 0 0
2019-03-04 11:25:00 20190304 GMKN 0 0 0
2019-03-04 11:30:00 20190304 GMKN 0 0 0
2019-03-04 11:35:00 20190304 GMKN 0 0 0
2019-03-04 11:40:00 20190304 GMKN 0 0 0


2019-03-04 18:20:00 20190304 LKOH 0 0 0
2019-03-04 18:25:00 20190304 LKOH 0 0 0
2019-03-04 18:30:00 20190304 LKOH 0 0 0
2019-03-04 18:35:00 20190304 LKOH 0 0 0
2019-03-04 18:40:00 20190304 LKOH 0 0 0
2019-03-04 18:45:00 20190304 LKOH 0 0 0
2019-03-04 10:05:00 20190304 MTSS 0 0 0
2019-03-04 10:10:00 20190304 MTSS 0 0 0
2019-03-04 10:15:00 20190304 MTSS 0 0 0
2019-03-04 10:20:00 20190304 MTSS 0 0 0
2019-03-04 10:25:00 20190304 MTSS 0 0 0
2019-03-04 10:30:00 20190304 MTSS 0 0 0
2019-03-04 10:35:00 20190304 MTSS 0 0 0
2019-03-04 10:40:00 20190304 MTSS 0 0 0
2019-03-04 10:45:00 20190304 MTSS 0 0 0
2019-03-04 10:50:00 20190304 MTSS 0 0 0
2019-03-04 10:55:00 20190304 MTSS 0 0 0
2019-03-04 11:00:00 20190304 MTSS 0 0 0
2019-03-04 11:05:00 20190304 MTSS 0 0 0
2019-03-04 11:10:00 20190304 MTSS 0 0 0
2019-03-04 11:15:00 20190304 MTSS 0 0 0
2019-03-04 11:20:00 20190304 MTSS 0 0 0
2019-03-04 11:25:00 20190304 MTSS 0 0 0
2019-03-04 11:30:00 20190304 MTSS 0 0 0
2019-03-04 11:35:00 20190304 MTSS 0 0 0


2019-03-04 18:35:00 20190304 MGNT 0 0 0
2019-03-04 18:40:00 20190304 MGNT 0 0 0
2019-03-04 18:45:00 20190304 MGNT 0 0 0
2019-03-04 10:05:00 20190304 TATN 0 0 0
2019-03-04 10:10:00 20190304 TATN 0 0 0
2019-03-04 10:15:00 20190304 TATN 0 0 0
2019-03-04 10:20:00 20190304 TATN 0 0 0
2019-03-04 10:25:00 20190304 TATN 0 0 0
2019-03-04 10:30:00 20190304 TATN 0 0 0
2019-03-04 10:35:00 20190304 TATN 0 0 0
2019-03-04 10:40:00 20190304 TATN 0 0 0
2019-03-04 10:45:00 20190304 TATN 0 0 0
2019-03-04 10:50:00 20190304 TATN 0 0 0
2019-03-04 10:55:00 20190304 TATN 0 0 0
2019-03-04 11:00:00 20190304 TATN 0 0 0
2019-03-04 11:05:00 20190304 TATN 0 0 0
2019-03-04 11:10:00 20190304 TATN 0 0 0
2019-03-04 11:15:00 20190304 TATN 0 0 0
2019-03-04 11:20:00 20190304 TATN 0 0 0
2019-03-04 11:25:00 20190304 TATN 0 0 0
2019-03-04 11:30:00 20190304 TATN 0 0 0
2019-03-04 11:35:00 20190304 TATN 0 0 0
2019-03-04 11:40:00 20190304 TATN 0 0 0
2019-03-04 11:45:00 20190304 TATN 0 0 0
2019-03-04 11:50:00 20190304 TATN 0 0 0


...

2019-03-29 14:40:00 20190329 NVTK 0 0 0
2019-03-29 14:45:00 20190329 NVTK 0 0 0
2019-03-29 14:50:00 20190329 NVTK 0 0 0
2019-03-29 14:55:00 20190329 NVTK 0 0 0
2019-03-29 15:00:00 20190329 NVTK 0 0 0
2019-03-29 15:05:00 20190329 NVTK 0 0 0
2019-03-29 15:10:00 20190329 NVTK 0 0 0
2019-03-29 15:15:00 20190329 NVTK 0 0 0
2019-03-29 15:20:00 20190329 NVTK 0 0 0
2019-03-29 15:25:00 20190329 NVTK 0 0 0
2019-03-29 15:30:00 20190329 NVTK 0 0 0
2019-03-29 15:35:00 20190329 NVTK 0 0 0
2019-03-29 15:40:00 20190329 NVTK 0 0 0
2019-03-29 15:45:00 20190329 NVTK 0 0 0
2019-03-29 15:50:00 20190329 NVTK 0 0 0
2019-03-29 15:55:00 20190329 NVTK 0 0 0
2019-03-29 16:00:00 20190329 NVTK 0 0 0
2019-03-29 16:05:00 20190329 NVTK 0 0 0
2019-03-29 16:10:00 20190329 NVTK 0 0 0
2019-03-29 16:15:00 20190329 NVTK 0 0 0
2019-03-29 16:20:00 20190329 NVTK 0 0 0
2019-03-29 16:25:00 20190329 NVTK 0 0 0
2019-03-29 16:30:00 20190329 NVTK 0 0 0
2019-03-29 16:35:00 20190329 NVTK 0 0 0
2019-03-29 16:40:00 20190329 NVTK 0 0 0


2019-03-29 15:35:00 20190329 ROSN 0 0 0
2019-03-29 15:40:00 20190329 ROSN 0 0 0
2019-03-29 15:45:00 20190329 ROSN 0 0 0
2019-03-29 15:50:00 20190329 ROSN 0 0 0
2019-03-29 15:55:00 20190329 ROSN 0 0 0
2019-03-29 16:00:00 20190329 ROSN 0 0 0
2019-03-29 16:05:00 20190329 ROSN 0 0 0
2019-03-29 16:10:00 20190329 ROSN 0 0 0
2019-03-29 16:15:00 20190329 ROSN 0 0 0
2019-03-29 16:20:00 20190329 ROSN 0 0 0
2019-03-29 16:25:00 20190329 ROSN 0 0 0
2019-03-29 16:30:00 20190329 ROSN 0 0 0
2019-03-29 16:35:00 20190329 ROSN 0 0 0
2019-03-29 16:40:00 20190329 ROSN 0 0 0
2019-03-29 16:45:00 20190329 ROSN 0 0 0
2019-03-29 16:50:00 20190329 ROSN 0 0 0
2019-03-29 16:55:00 20190329 ROSN 0 0 0
2019-03-29 17:00:00 20190329 ROSN 0 0 0
2019-03-29 17:05:00 20190329 ROSN 0 0 0
2019-03-29 17:10:00 20190329 ROSN 0 0 0
2019-03-29 17:15:00 20190329 ROSN 0 0 0
2019-03-29 17:20:00 20190329 ROSN 0 0 0
2019-03-29 17:25:00 20190329 ROSN 0 0 0
2019-03-29 17:30:00 20190329 ROSN 0 0 0
2019-03-29 17:35:00 20190329 ROSN 0 0 0


2019-03-29 16:50:00 20190329 VTBR 0 0 0
2019-03-29 16:55:00 20190329 VTBR 0 0 0
2019-03-29 17:00:00 20190329 VTBR 0 0 0
2019-03-29 17:05:00 20190329 VTBR 0 0 0
2019-03-29 17:10:00 20190329 VTBR 0 0 0
2019-03-29 17:15:00 20190329 VTBR 0 0 0
2019-03-29 17:20:00 20190329 VTBR 0 0 0
2019-03-29 17:25:00 20190329 VTBR 0 0 0
2019-03-29 17:30:00 20190329 VTBR 0 0 0
2019-03-29 17:35:00 20190329 VTBR 0 0 0
2019-03-29 17:40:00 20190329 VTBR 0 0 0
2019-03-29 17:45:00 20190329 VTBR 0 0 0
2019-03-29 17:50:00 20190329 VTBR 0 0 0
2019-03-29 17:55:00 20190329 VTBR 0 0 0
2019-03-29 18:00:00 20190329 VTBR 0 0 0
2019-03-29 18:05:00 20190329 VTBR 0 0 0
2019-03-29 18:10:00 20190329 VTBR 0 0 0
2019-03-29 18:15:00 20190329 VTBR 0 0 0
2019-03-29 18:20:00 20190329 VTBR 0 0 0
2019-03-29 18:25:00 20190329 VTBR 0 0 0
2019-03-29 18:30:00 20190329 VTBR 0 0 0
2019-03-29 18:35:00 20190329 VTBR 0 0 0
2019-03-29 18:40:00 20190329 VTBR 0 0 0
2019-03-29 18:45:00 20190329 VTBR 0 0 0
2019-03-29 10:05:00 20190329 SNGS 0 0 0


2019-03-29 16:45:00 20190329 CHMF 0 0 0
2019-03-29 16:50:00 20190329 CHMF 0 0 0
2019-03-29 16:55:00 20190329 CHMF 0 0 0
2019-03-29 17:00:00 20190329 CHMF 0 0 0
2019-03-29 17:05:00 20190329 CHMF 0 0 0
2019-03-29 17:10:00 20190329 CHMF 0 0 0
2019-03-29 17:15:00 20190329 CHMF 0 0 0
2019-03-29 17:20:00 20190329 CHMF 0 0 0
2019-03-29 17:25:00 20190329 CHMF 0 0 0
2019-03-29 17:30:00 20190329 CHMF 0 0 0
2019-03-29 17:35:00 20190329 CHMF 0 0 0
2019-03-29 17:40:00 20190329 CHMF 0 0 0
2019-03-29 17:45:00 20190329 CHMF 0 0 0
2019-03-29 17:50:00 20190329 CHMF 0 0 0
2019-03-29 17:55:00 20190329 CHMF 0 0 0
2019-03-29 18:00:00 20190329 CHMF 0 0 0
2019-03-29 18:05:00 20190329 CHMF 0 0 0
2019-03-29 18:10:00 20190329 CHMF 0 0 0
2019-03-29 18:15:00 20190329 CHMF 0 0 0
2019-03-29 18:20:00 20190329 CHMF 0 0 0
2019-03-29 18:25:00 20190329 CHMF 0 0 0
2019-03-29 18:30:00 20190329 CHMF 0 0 0
2019-03-29 18:35:00 20190329 CHMF 0 0 0
2019-03-29 18:40:00 20190329 CHMF 0 0 0
2019-03-29 18:45:00 20190329 CHMF 0 0 0


,TIME,DATE,SECCODE,FLAG,ICE_COUNT
0,2019-03-04 10:05:00,20190304,GAZP,0,0
1,2019-03-04 10:10:00,20190304,GAZP,0,0
2,2019-03-04 10:15:00,20190304,GAZP,0,0
3,2019-03-04 10:20:00,20190304,GAZP,0,0
4,2019-03-04 10:25:00,20190304,GAZP,0,0
...,...,...,...,...,...
29920,2019-03-29 18:25:00,20190329,ALRS,0,0
29921,2019-03-29 18:30:00,20190329,ALRS,0,0
29922,2019-03-29 18:35:00,20190329,ALRS,0,0
29923,2019-03-29 18:40:00,20190329,ALRS,0,0


In [38]:
ice_per_5min1.dtypes

TIME         datetime64[ns]
DATE                 object
SECCODE              object
FLAG                  int64
ICE_COUNT             int64
dtype: object

In [51]:
liquidity1.dtypes

TIME             datetime64[ns]
DATE                     object
SECCODE                  object
BID_PRICE               float64
BID_VOLUME                int64
ASK_PRICE               float64
ASK_VOLUME                int64
SPREAD                  float64
together_time            object
dtype: object

# (14.12) Объединяем итоги группировки айсберг-заявок и вычисленных состояний стакана

In [52]:
%%time
liquidity2 = liquidity1.merge(ice_per_5min1, on = ['SECCODE', 'TIME', 'DATE'])

liquidity2

CPU times: user 34.5 ms, sys: 4.64 ms, total: 39.1 ms
Wall time: 50 ms


,TIME,DATE,SECCODE,BID_PRICE,BID_VOLUME,ASK_PRICE,ASK_VOLUME,SPREAD,together_time,FLAG,ICE_COUNT
0,2019-03-04 10:05:00,20190304,GAZP,156.38,610,156.40,2880,0.02,2019-03-04 10:05:00,0,0
1,2019-03-04 10:10:00,20190304,GAZP,156.26,22160,156.30,800,0.04,2019-03-04 10:10:00,0,0
2,2019-03-04 10:15:00,20190304,GAZP,156.36,10,156.40,2290,0.04,2019-03-04 10:15:00,0,0
3,2019-03-04 10:20:00,20190304,GAZP,156.10,2210,156.16,2430,0.06,2019-03-04 10:20:00,0,0
4,2019-03-04 10:25:00,20190304,GAZP,156.30,410,156.36,460,0.06,2019-03-04 10:25:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
29920,2019-03-29 18:25:00,20190329,ALRS,92.27,10,92.28,19150,0.01,2019-03-29 18:25:00,0,0
29921,2019-03-29 18:30:00,20190329,ALRS,92.49,3020,92.51,1320,0.02,2019-03-29 18:30:00,0,0
29922,2019-03-29 18:35:00,20190329,ALRS,92.58,15410,92.59,34590,0.01,2019-03-29 18:35:00,0,0
29923,2019-03-29 18:40:00,20190329,ALRS,92.56,1240,92.59,4500,0.03,2019-03-29 18:40:00,0,0


In [53]:
liquidity1.to_csv('liquidity1.csv', index = False)

In [55]:
test1 = pd.read_csv('/Users/ekaterina/Desktop/w/курсовая (2)/data/updated/OrderLog20190304.txt', header = 0)
test1.head(50)

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,AFKS,S,100000000000,1,1,10.05700,100,NaN,NaN
1,2,ABRD,S,100000000000,2,1,150.00000,500,NaN,NaN
2,3,ABRD,S,100000000000,3,1,139.50000,30,NaN,NaN
3,4,AFLT,S,100000000000,4,1,102.74000,10,NaN,NaN
4,5,AGRO,B,100000000000,5,1,786.00000,1,NaN,NaN
5,6,AFKS,B,100000000000,6,1,9.19200,100,NaN,NaN
6,7,AFKS,S,100000000000,7,1,10.43700,100,NaN,NaN
7,8,AFKS,S,100000000000,8,1,10.49700,10000,NaN,NaN
8,9,AFKS,S,100000000000,9,1,10.32000,200,NaN,NaN
9,10,BANEP,S,100000000000,10,1,1892.00000,1,NaN,NaN
